In [2]:
using LinearAlgebra;
using StaticArrays;
using Makie
using Makie:Point2f0
using CairoMakie
using Printf
using Symbolics


## Construction of the Hamiltonian -- Both spin states are included in the lattice Hamiltonian

In [6]:
# Here, we define the three vectors connecting nearest neighbor Kagome atoms 

# Useful C_3 rotation
global const C_3::Matrix{Float64} = [cos(2.0*pi/3.0) -sin(2.0*pi/3.0); sin(2.0*pi/3.0) cos(2.0*pi/3.0)]

# Nearest neighbor vectors 
global const A1::Vector{Float64} = [1.0, 0.0]
global const A2::Vector{Float64} = [0.5, 0.5 * sqrt(3.0)]
global const A3::Vector{Float64} = A2 - A1; 


# Useful in constructing the Kagome Hamiltonian 
global const T1u  = @SArray ComplexF64[0.0 1.0 0.0; 0.0 0.0 0.0; 0.0 0.0 0.0]
global const T2u  = @SArray ComplexF64[0.0 0.0 1.0; 0.0 0.0 0.0; 0.0 0.0 0.0]
global const T3u  = @SArray ComplexF64[0.0 0.0 0.0; 0.0 0.0 1.0; 0.0 0.0 0.0];

# Reciprocal and Direct Lattice 
global const R1::Vector{Float64} = 2.0 * A1 
global const R2::Vector{Float64} = 2.0 * A2

global const R::Matrix{Float64} = [R1 R2]';
global const G::Matrix{Float64} = 2.0 * pi * inv(R');

# Kagome Model
@inline function H_Kagome(k::Vector{Float64}, t1::Float64, t2::Float64, u1::Float64, u2::Float64)
    Φ1::ComplexF64 = 2.0 * ((-t1 + im * u1) * cos(kx) + (-t2 + im * u2) * cos(sqrt(3.0) * ky)) 
    Φ2::ComplexF64 = 2.0 * ((-t1 - im * u1) * cos(0.5 * kx + 0.5 * sqrt(3.0) * ky) + (-t2 + im * u2) * cos(-0.5 * 3.0 * kx + 0.5 * sqrt(3.0) * ky)) 
    Φ3::ComplexF64 = 2.0 * ((-t1 + im * u1) * cos(-0.5 * kx + 0.5 * sqrt(3.0) * ky) + (-t2 - im * u2) * cos(0.5 * 3.0 * kx + 0.5 * sqrt(3.0) * ky)) 

    Φ1 * T1u + Φ2 * T2u + Φ3 * T3u
 end

 @inline function evals(k::Vector{Float64}, t1::Float64, t2::Float64, u1::Float64, u2::Float64)
   kx::Float64 = k[1]; ky::Float64 = k[2]
   Φ1::ComplexF64 = 2.0 * ((-t1 + im * u1) * cos(kx) + (-t2 + im * u2) * cos(sqrt(3.0) * ky)) 
   Φ2::ComplexF64 = 2.0 * ((-t1 - im * u1) * cos(0.5 * kx + 0.5 * sqrt(3.0) * ky) + (-t2 + im * u2) * cos(-0.5 * 3.0 * kx + 0.5 * sqrt(3.0) * ky)) 
   Φ3::ComplexF64 = 2.0 * ((-t1 + im * u1) * cos(-0.5 * kx + 0.5 * sqrt(3.0) * ky) + (-t2 - im * u2) * cos(0.5 * 3.0 * kx + 0.5 * sqrt(3.0) * ky)) 

   H::Matrix{ComplexF64} = Φ1 * T1u + Φ2 * T2u + Φ3 * T3u
   eigvals(Hermitian(H + H'))
end


@inline function eigensystem_kagome(k::Vector{Float64}, t1::Float64, t2::Float64, u1::Float64, u2::Float64)
   kx::Float64 = k[1]; ky::Float64 = k[2]
   Φ1::ComplexF64 = 2.0 * ((-t1 + im * u1) * cos(kx) + (-t2 + im * u2) * cos(sqrt(3.0) * ky)) 
   Φ2::ComplexF64 = 2.0 * ((-t1 - im * u1) * cos(0.5 * kx + 0.5 * sqrt(3.0) * ky) + (-t2 + im * u2) * cos(-0.5 * 3.0 * kx + 0.5 * sqrt(3.0) * ky)) 
   Φ3::ComplexF64 = 2.0 * ((-t1 + im * u1) * cos(-0.5 * kx + 0.5 * sqrt(3.0) * ky) + (-t2 - im * u2) * cos(0.5 * 3.0 * kx + 0.5 * sqrt(3.0) * ky)) 

   H::Matrix{ComplexF64} = Φ1 * T1u + Φ2 * T2u + Φ3 * T3u
   eigen(Hermitian(H + H'))
end


@inline function Vx(k::Vector{Float64}, t1::Float64, t2::Float64, u1::Float64, u2::Float64)
    kx::Float64 = k[1]; ky::Float64 = k[2]
    dΦ1_dx::ComplexF64 = -2.0 * ((-t1 + im * u1) * sin(kx))
    dΦ2_dx::ComplexF64 = 2.0 * (-(-t1 - im * u1) * sin(0.5 * kx + 0.5 * sqrt(3.0) * ky) + (-t2 + im * u2) * 1.5 * sin(-0.5 * 3.0 * kx + 0.5 * sqrt(3.0) * ky)) 
    dΦ3_dx::ComplexF64 = 2.0 * (0.5 * (-t1 + im * u1) * sin(-0.5 * kx + 0.5 * sqrt(3.0) * ky) - 1.5 * (-t2 - im * u2) * sin(0.5 * 3.0 * kx + 0.5 * sqrt(3.0) * ky)) 
    m::Matrix{ComplexF64} = dΦ1_dx * T1u + dΦ2_dx * T2u + dΦ3_dx * T3u
    m + m'
end

@inline function Vy(k::Vector{Float64}, t1::Float64, t2::Float64, u1::Float64, u2::Float64)
    kx::Float64 = k[1]; ky::Float64 = k[2]
    dΦ1_dy::ComplexF64 = -sqrt(3.0) * (-t2 + im * u2) * sin(sqrt(3.0) * ky)
    dΦ2_dy::ComplexF64 = sqrt(3.0) * (-(-t1 - im * u1) * sin(0.5 * kx + 0.5 * sqrt(3.0) * ky) - (-t2 + im * u2) * sin(-0.5 * 3.0 * kx + 0.5 * sqrt(3.0) * ky)) 
    dΦ3_dy::ComplexF64 = sqrt(3.0) * (-(-t1 + im * u1) * sin(-0.5 * kx + 0.5 * sqrt(3.0) * ky) - (-t2 - im * u2) * sin(0.5 * 3.0 * kx + 0.5 * sqrt(3.0) * ky)) 
    m::Matrix{ComplexF64} = dΦ1_dy * T1u + dΦ2_dy * T2u + dΦ3_dy * T3u
    m + m'
end;

@inline function calculate_kgrid(; G::Matrix{Float64} = G, nps::Int = 400)
    """
    Calculate a regular grid in the FBZ 
    """
    G1::Vector{Float64} = G[1, :]; G2::Vector{Float64} = G[2, :]
    kgrid = Matrix{Vector{Float64}}(undef, nps, nps)
    for n ∈ 1 : nps
        for m ∈ 1 : nps
            kgrid[n, m] = ((n - 1) * G1 + (m - 1) * G2) / nps
        end
    end
    kgrid
end

@inline function Berry_Curvature(k::Vector{Float64},
    t1::Float64, 
    t2::Float64, 
    u1::Float64, 
    u2::Float64, 
    band::Int = 1)

    (E, U) = eigensystem_kagome(k, t1, t2, u1, u2)
    Vxb::Matrix{ComplexF64} = U' * Vx(k, t1, t2, u1, u2) * U
    Vyb::Matrix{ComplexF64} = U' * Vy(k, t1, t2, u1, u2) * U

    bck::Float64 = 0.0

    for n ∈ 1:3
        if (n != band)
            bck += 2.0 * imag(Vxb[band, n] * Vyb[n, band]) / (E[band] - E[n])^2
        end
    end
    return bck
end

@inline function Quantum_Metric_xy(k::Vector{Float64},
    t1::Float64, 
    t2::Float64, 
    u1::Float64, 
    u2::Float64, 
    band::Int = 1)

    (E, U) = eigensystem_kagome(k, t1, t2, u1, u2)
    Vxb::Matrix{ComplexF64} = U' * Vx(k, t1, t2, u1, u2) * U
    Vyb::Matrix{ComplexF64} = U' * Vy(k, t1, t2, u1, u2) * U

    bck::Float64 = 0.0

    for n ∈ 1:3
        if (n != band)
            bck += real(Vxb[band, n] * Vyb[n, band]) / (E[band] - E[n])^2
        end
    end
    return bck
end

@inline function Quantum_Metric_xx(k::Vector{Float64},
    t1::Float64, 
    t2::Float64, 
    u1::Float64, 
    u2::Float64, 
    band::Int = 1)

    (E, U) = eigensystem_kagome(k, t1, t2, u1, u2)
    Vxb::Matrix{ComplexF64} = U' * Vx(k, t1, t2, u1, u2) * U
    
    bck::Float64 = 0.0

    for n ∈ 1:3
        if (n != band)
            bck += real(Vxb[band, n] * Vxb[n, band]) / (E[band] - E[n])^2
        end
    end
    return bck
end

@inline function Quantum_Metric_yy(k::Vector{Float64},
    t1::Float64, 
    t2::Float64, 
    u1::Float64, 
    u2::Float64, 
    band::Int = 1)

    (E, U) = eigensystem_kagome(k, t1, t2, u1, u2)
    Vxb::Matrix{ComplexF64} = U' * Vy(k, t1, t2, u1, u2) * U
    
    bck::Float64 = 0.0

    for n ∈ 1:3
        if (n != band)
            bck += real(Vyb[band, n] * Vyb[n, band]) / (E[band] - E[n])^2
        end
    end
    return bck
end


Quantum_Metric_yy (generic function with 2 methods)